In [1]:
#ARVIS TECH Internship Programme
#September 2022

#liveness detection

In [2]:
import cv2
import mediapipe as mp
import numpy as np
from scipy.signal import filtfilt, butter, lfilter, medfilt
import math
import pandas as pd
from scipy import stats

In [3]:
#loading data to pandas dataframe
df = pd.read_csv('veri.csv')
df

,name,SNR,STD,powerSNR,powerSTD,dxSNR,dxSTD,isReal
0,Ali_Tarkan_1,52.796504,2.581225,0.232198,0.000419,-0.007777,0.258429,1
1,Ali_Tarkan_2,77.553153,1.943144,0.230828,0.000652,0.029859,0.255824,1
2,Ali_Tarkan_3,116.801788,1.136381,0.175520,0.002281,-0.013913,0.225860,1
3,Ali_Tarkan_4,227.252392,0.634131,0.213033,0.008063,-0.003158,0.303882,1
4,Ali_Tarkan_5,115.329986,1.078965,0.254745,0.000690,0.006837,0.148160,1
...,...,...,...,...,...,...,...,...
120,Esra_7,95.865000,1.506816,0.257095,0.000550,-0.028334,0.267591,1
121,Esra_8,97.236704,1.464426,0.240360,0.001375,-0.018673,0.259286,1
122,Esra_9,102.787606,1.569953,0.242229,0.001603,0.001045,0.431196,1
123,Esra_10,98.181252,1.353960,0.259275,0.001051,-0.019801,0.274047,1


In [4]:
#selecting data and target columns
target_name = "isReal"
target = df[target_name]
data = df.drop(columns=[target_name, "name"])
data

,SNR,STD,powerSNR,powerSTD,dxSNR,dxSTD
0,52.796504,2.581225,0.232198,0.000419,-0.007777,0.258429
1,77.553153,1.943144,0.230828,0.000652,0.029859,0.255824
2,116.801788,1.136381,0.175520,0.002281,-0.013913,0.225860
3,227.252392,0.634131,0.213033,0.008063,-0.003158,0.303882
4,115.329986,1.078965,0.254745,0.000690,0.006837,0.148160
...,...,...,...,...,...,...
120,95.865000,1.506816,0.257095,0.000550,-0.028334,0.267591
121,97.236704,1.464426,0.240360,0.001375,-0.018673,0.259286
122,102.787606,1.569953,0.242229,0.001603,0.001045,0.431196
123,98.181252,1.353960,0.259275,0.001051,-0.019801,0.274047


In [5]:
target

0      1
1      1
2      1
3      1
4      1
      ..
120    1
121    1
122    1
123    1
124    1
Name: isReal, Length: 125, dtype: int64

In [6]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()

In [7]:
from sklearn.model_selection import cross_validate
cv_result = cross_validate(model, data, target, cv=5)
cv_result

{'fit_time': array([0.10570669, 0.11027384, 0.12271118, 0.10292506, 0.11517906]),
 'score_time': array([0.00897622, 0.01187611, 0.01097608, 0.00895786, 0.01095366]),
 'test_score': array([0.84, 0.72, 0.72, 0.88, 0.84])}

In [8]:
scores = cv_result["test_score"]
print(
    "The mean cross-validation accuracy is: "
    f"{scores.mean():.3f} +/- {scores.std():.3f}"
)

The mean cross-validation accuracy is: 0.800 +/- 0.067


In [9]:
model.fit(data.values, target.values)

RandomForestClassifier()

In [10]:
def polygon(image, coordinates, facial_landmarks):
    #returns the region in the frame as a polygon for the selected landmark points.
    
    height, width, _ = image.shape

    area = []
    for i in coordinates:
        pt1 = facial_landmarks.landmark[i]
        x = int(pt1.x * width)
        y = int(pt1.y * height)
        area.append([x,y])
        #cv2.circle(image, (x, y), 2, (100, 100, 0), -1)
        #cv2.putText(image, str(i), (x, y), 0, 0.5, (0, 0, 0))
        
    area = np.array(area)
    area = area.reshape((-1, 1, 2))
    #cv2.polylines(image,[area],True,(0,255,255))
    
    return area

In [11]:
def extract_roi(image, yanak_sag , yanak_sol, alin):
    #this function extracts ROI areas and put them on a black background from the base image
    
    mask = np.zeros(image.shape[:2], dtype="uint8")
    cv2.drawContours(mask, [yanak_sag], -1, (255, 255, 255), -1, cv2.LINE_AA)
    cv2.drawContours(mask, [yanak_sol], -1, (255, 255, 255), -1, cv2.LINE_AA)
    cv2.drawContours(mask, [alin], -1, (255, 255, 255), -1, cv2.LINE_AA)

    output = cv2.bitwise_and(image, image, mask=mask)
    return output

In [12]:
def green(image, area):
    #function to find green channel averages of given image and ROI
    
    mask_G = np.zeros(image.shape[:2], dtype="uint8")
    cv2.drawContours(mask_G, [area], -1, (255, 255, 255), -1, cv2.LINE_AA)
    mean = cv2.mean(image[:,:,1], mask=mask_G)
    #print(f"{txt}= {mean}")
    return mean

In [13]:
def update_signal(green_mean, green_arr, n_frames):
    #updates green channel array signal
    
    #if size of array is less than given limit, it appends the green channel mean to the signal array
    if len(green_arr) < n_frames:
        green_arr.append(green_mean)
        
    #else it keeps the size of the array constant by deleting an element from the beginning and appending an element to the end
    else:
        del green_arr[0]
        green_arr.append(green_mean)

In [14]:
#Function to find SNR values of green_channel array
def signaltonoise(a, axis=0, ddof=0):
    a = np.asanyarray(a)
    m = a.mean(axis)
    sd = a.std(axis=axis, ddof=ddof)
    return np.where(sd == 0, 0, m/sd)

In [15]:
#finds power spectrum of an array
def power_spectrum(mean_list, fps, hz_bottom, hz_top):
    sampling_rate = fps
    raw_signal = np.squeeze(mean_list) 
    normalized_signal = raw_signal - np.mean(raw_signal)
    normalized_signal = normalized_signal / np.std(normalized_signal)
    med_signal = medfilt(normalized_signal, 5)
    nyq = 0.5 * sampling_rate
    low = hz_bottom / nyq
    high = hz_top / nyq

    b, a = butter(3, [low, high], btype = 'band')
    filtered_signal = lfilter(b, a, med_signal)
    window = np.hamming(len(filtered_signal))
    windowed_signal = filtered_signal * window
    def nextpow(x):
        return 2 ** math.ceil(math.log2(x))

    NFFT = nextpow(len(windowed_signal) * sampling_rate)
    freqDomain = np.fft.fft(windowed_signal, NFFT)
    conj = np.conjugate(freqDomain)

    powerSpectrum = np.multiply(freqDomain, conj) / NFFT
    powerSpectrum = np.real(powerSpectrum)
    return powerSpectrum

In [16]:
def pre_train(array, fps):
    #this function returns (SNR, STD, SNR of Power Spectrum, STD of Power Spectrum,
    #..SNR of the derivative, STD of the derivative) values of the given signal array
    #these values will be used in classification model.
    
    snr = signaltonoise(array)
    std = np.std(array)
    ps = power_spectrum(array, fps, 1.0, 2.75)
    powerSNR = signaltonoise(ps)
    powerSTD = np.std(ps)
    dx = np.diff(array)
    dxSNR = signaltonoise(dx)
    dxSTD = np.std(dx)
     
    return [float(snr), std, float(powerSNR), powerSTD, float(dxSNR), dxSTD]
    

In [17]:
def start_video():
    
    #video input
    cap = cv2.VideoCapture(r"C:\Users\ahmet\OneDrive\Masaüstü\yüz videoları\videolar\Validation\Test_19.mp4")
    
    #finds the fps of the video
    (major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')
    if int(major_ver)  < 3 :
        fps = cap.get(cv2.cv.CV_CAP_PROP_FPS)
    else :
        fps = cap.get(cv2.CAP_PROP_FPS)
    

    font = cv2.FONT_HERSHEY_SIMPLEX 

    # Face Mesh
    mp_face_mesh = mp.solutions.face_mesh
    face_mesh = mp_face_mesh.FaceMesh()

    #array to store green channel means of each frame   
    green_arr = [] 
    
    #frame counter
    ctr = 0 
    while True:
        ret, image = cap.read()
        if ret is not True:
            break


        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # Facial landmarks
        result = face_mesh.process(rgb_image)
        
        #mediapipe crashes in case it cannot find a face to detect therefore try/except blocks used here,
        try:

            for facial_landmarks in result.multi_face_landmarks:
                
                #left cheek landmark points 
                yanak_sol_arr = [143, 123, 147, 213, 138, 135, 210, 202, 57, 92, 36, 101, 118, 31, 143]
                yanak_sol = polygon(image, yanak_sol_arr, facial_landmarks)

            for facial_landmarks in result.multi_face_landmarks:

                #right cheek landmark points 
                yanak_sag_arr = [372, 352, 376, 433, 367, 364, 430, 422, 287, 322, 266, 330, 347, 261, 372]
                yanak_sag = polygon(image, yanak_sag_arr, facial_landmarks)

            for facial_landmarks in result.multi_face_landmarks:

                #forehead landmark points 
                alin_arr = [54, 103, 67, 109, 10, 338, 297, 332, 284, 298, 296, 9, 66, 68,]
                alin = polygon(image, alin_arr, facial_landmarks)
                
                        
            #extracts ROI areas from the base image
            image = extract_roi(image, yanak_sag , yanak_sol, alin)

            #the green channel average of each ROI areas
            mean_yanak_sag = green(image, yanak_sag)[0]
            mean_yanak_sol = green(image, yanak_sol)[0]
            mean_alin = green(image, alin)[0]

            #mean green channel value of all ROI areas combined
            mean_raw = (mean_yanak_sag + mean_yanak_sol + mean_alin)/3
            
            #updates green channel array
            #the limit of the array is 30 seconds.
            #The reason we chose 30 seconds because all the videos used for training the model were recorded as 30 seconds
            update_signal(mean_raw, green_arr, n_frames=fps*30)
            

        #if mediapipe cannot detects face it prints "No Face Detected" to the screen
        except:
            text = "No Face Detected"
            coordinates = (128,128)
            fontScale = 1
            color = (255,0,255)
            thickness = 2
            blank = np.zeros(image.shape[:2], dtype="uint8")
            image = cv2.putText(blank, text, coordinates, font, fontScale, color, thickness, cv2.LINE_AA)

        cv2.imshow("canlilik", image)
        
        #increase counter
        ctr += 1
    
        if cv2.waitKey(1) == ord("q"):
            break
    
    #Finding the attributes of the array to give to the machine learning algorithm
    liveness_arr = pre_train(green_arr, fps)
    
    #predicting; 0 for fake videos, 1 for real videos
    sonuc = model.predict([liveness_arr])
    
    #prints the result
    if sonuc:
        print("Canli")
    else:
        print("Cansiz")

    cap.release()
    cv2.destroyAllWindows()

In [18]:
start_video()

Cansiz
